In [9]:
X1 = read.csv("datafile/X1.csv")
X2 = read.csv("datafile/X2.csv")
X3 = read.csv("datafile/X3.csv")
X4 = read.csv("datafile/X4.csv")
X5 = read.csv("datafile/X5.csv")
X6 = read.csv("datafile/X6.csv")
X7 = read.csv("datafile/X7.csv")
X8 = read.csv("datafile/X8.csv")
X9 = read.csv("datafile/X9.csv")
X10 = read.csv("datafile/X10.csv")

SM = read.csv("datafile/SM.csv")
TC = read.csv("datafile/TC.csv")
options(repr.matrix.max.cols=150, repr.matrix.max.rows=200)

In [10]:
SM <- data.matrix(SM)
SM <- SM[, colnames(SM) != "X"]

TC <- data.matrix(TC)
TC <- TC[, colnames(TC) != "X"]

In [11]:
# Setting up rhos
r <- seq(from = 0, to = 1, by = 0.05)

# Setting up constant variables
N <- 240
nsrcs <- 6
x1 <- 21
x2 <- 21
V <- 441

In [12]:
#Lasso Regression Function
LassoReg <- function(X, thr) {  
    X <- data.matrix(X)
    X <- X[, colnames(X) != "X"]
    for ( k in 1 : ( x1*x2 ) ) {
        A <- Ao + step *( t(TC)%*%(X[ , k] - (TC%*%Ao ) ) )
        A <- (1/(1+thr ) ) * (sign(A)*pmax(replicate(nsrcs, 0 ) , abs(A) - thr ) )
        for ( i in 1:10 ) {
            Ao <- A
            A <- Ao+step *( t(TC)%*%(X[ , k] - (TC%*%Ao ) ) )
            A <- (1/(1+thr)) * (sign(A)*pmax(replicate(nsrcs , 0 ) , abs(A) -thr ) )
        }
        Alr[ , k] <- A
    }
    return (Alr)
}

In [13]:
SMSE <- c()

# Find average MSE for 21 different rhos
for (rho in r) {

    # Setting up variables for Lasso Regression
    step <- 1/(norm(TC%*%t(TC) ) * 1.1 )
    thr <- rho * N * step
    Ao <- matrix( 0 , nsrcs , 1 )
    A <- matrix( 0 , nsrcs , 1 )
    Alr <- matrix( 0 , nsrcs , x1*x2 )

    
    # Finding Alr for all different standardised Xs (10 realisations)
    Alr1 <- LassoReg(X1, thr)
    Alr2 <- LassoReg(X2, thr)
    Alr3 <- LassoReg(X3, thr)
    Alr4 <- LassoReg(X4, thr)
    Alr5 <- LassoReg(X5, thr)
    Alr6 <- LassoReg(X6, thr)
    Alr7 <- LassoReg(X7, thr)
    Alr8 <- LassoReg(X8, thr)
    Alr9 <- LassoReg(X9, thr)
    Alr10 <- LassoReg(X10, thr)



    
    alr_list <- list(Alr1, Alr2, Alr3, Alr4, Alr5, Alr6, Alr7, Alr8, Alr9, Alr10)
    X_list <- list(X1, X2, X3, X4, X5, X6, X7, X8, X9, X10)

    # Find average MSE over 10 realisations
    sum_MSE = 0
    for (i in (1:10)) {
        Alr <- alr_list[[i]]
        X <- X_list[[i]]
        X <- data.matrix(X)
        X <- X[,-1]
        D_lr <- (X %*% t(Alr))
        sum_MSE <- sum_MSE + (sum(sum((X - (D_lr %*% Alr))^2)) / (N*V))
    }

    avg_SMSE <- sum_MSE / 10
    SMSE <- append(SMSE, avg_SMSE)
}

In [14]:
write.csv(SMSE,file="datafile/smse.csv", row.names=FALSE)

In [15]:
min_rho <- 0.7
thr <- min_rho * N * step
X <- read.csv("datafile/X.csv")
X <- data.matrix(X)
X <- X[,-1]
Alr <- LassoReg(X, thr)
D_lr <- (X %*% t(Alr))

In [16]:
write.csv(Alr,file="datafile/alr.csv", row.names=FALSE)
write.csv(D_lr,file="datafile/dlr.csv", row.names=FALSE)